In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [2]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Busselton,-33.6500,115.3333,56.97,76,61,3.71,AU,2022-10-20 23:12:45
1,1,Mataura,-46.1927,168.8643,50.14,41,71,5.91,NZ,2022-10-20 23:17:10
2,2,Hilo,19.7297,-155.0900,88.29,70,0,12.66,US,2022-10-20 23:14:34
3,3,Diffa,13.3154,12.6113,84.79,21,92,8.41,NE,2022-10-20 23:17:11
4,4,Hermanus,-34.4187,19.2345,56.66,85,68,9.55,ZA,2022-10-20 23:17:12


In [3]:
# Get the data types.
city_data_df.dtypes

City_ID         int64
City           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
Country        object
Date           object
dtype: object

In [4]:
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))

In [5]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]

# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]

# Assign the figure variable.
fig = gmaps.figure(center=(30.0, 31.0), 
                   zoom_level=1.5)

# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, 
                                 weights=[max(temp, 0) for temp in max_temp], 
                                 dissipating=False, 
                                 max_intensity=300, 
                                 point_radius=4)

# Add the heatmap layer.
fig.add_layer(heat_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [6]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), 
                   zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, 
                                 weights=humidity, 
                                 dissipating=False, 
                                 max_intensity=300, 
                                 point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [7]:
# Heatmap of percent cloudiness
locations = city_data_df[["Lat", "Lng"]]
clouds = city_data_df["Cloudiness"]
fig = gmaps.figure(center=(30.0, 31.0), 
                   zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, 
                                 weights=clouds, 
                                 dissipating=False, 
                                 max_intensity=300, 
                                 point_radius=4)
fig.add_layer(heat_layer)

# Call the figure to plot the data
fig

Figure(layout=FigureLayout(height='420px'))

In [8]:
# Heatmap of wind speed
locations = city_data_df[["Lat", "Lng"]]
wind = city_data_df["Wind Speed"]
fig = gmaps.figure(center=(30.0, 31.0), 
                   zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, 
                                 weights=wind, 
                                 dissipating=False, 
                                 max_intensity=300, 
                                 point_radius=4)
fig.add_layer(heat_layer)

# Call the figure to plot the data
fig

Figure(layout=FigureLayout(height='420px'))

In [9]:
#6.5.3

In [10]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [11]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & 
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.count()

City_ID       172
City          172
Lat           172
Lng           172
Max Temp      172
Humidity      172
Cloudiness    172
Wind Speed    172
Country       172
Date          172
dtype: int64

In [12]:
#6.5.4

In [13]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
2,Hilo,US,88.29,19.7297,-155.0900,
3,Diffa,NE,84.79,13.3154,12.6113,
7,Salinopolis,BR,81.21,-0.6136,-47.3561,
13,Sarh,TD,75.87,9.1429,18.3923,
15,Adrar,MR,88.50,20.5022,-10.0711,
16,Atambua,ID,79.48,-9.1061,124.8925,
24,Kapaa,US,77.09,22.0752,-159.3190,
27,Guerrero Negro,MX,78.31,27.9769,-114.0611,
28,Damaturu,NG,81.41,11.7470,11.9608,
30,Acapulco,MX,75.63,16.8634,-99.8901,


In [14]:
# Set parameters to search for a hotel.
params = {
    "radius": 1500,
    "type": "hotel",
    "keyword": "hotel",
    "key": g_key
}

In [ ]:
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    
    # extract results
    results = hotels['results']
    
    try:
        print(f"{results[0]['name']} found.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Hotel not found... skipping.")
        
    print("------------")


Hilo Bay Hostel found.
------------
Hotel not found... skipping.
------------
Hotel Solar found.
------------
Hotel not found... skipping.
------------
Hotel not found... skipping.
------------
Hotel King Star found.
------------
Hotel Coral Reef found.
------------
Hotel not found... skipping.
------------
GAAT HOTEL found.
------------
Hotel Krystal Beach Acapulco found.
------------


In [ ]:
hotel_df.head(10)

In [ ]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

In [ ]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

In [ ]:
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]